# PS Injection Bump Tune Swing: Power Supply Ripple

In [4]:
import imageio
import pickle
import datetime
import pandas as pd
import numpy as np
import PyNAFF as pnf
import scipy.io as sio 
import matplotlib.cm as cm
from math import log10, floor
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from scipy.optimize import curve_fit

## Make the BSW kicker tables

In [5]:

half_bump = True

bsw40 = -1*0.003498034648
bsw42 = -1*-0.01422287844
bsw43 = -1*0.01290524313
bsw44 = -1*-0.006001526439
BSS_max = 2*1.11828e-01 #from twiss 

f = open("BSEXT_Bump_SBEND.tfs","w")

# build the parameter sweep lists
nstep = 100.

t = np.arange(0,nstep+1,1)/nstep * np.pi/2
t = np.append(t, t+np.pi/2)

# Half the values for only bump closure
len_t = len(t)

def half_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

s = half_list(t)[1]
u = half_list(t)[0]

# strengths for the simulation

# sextupole strength
BSS_val = BSS_max*np.cos(t)

# dipole (kicker) strength
bsw40_val = bsw40*np.sin(t)
bsw42_val = bsw42*np.sin(t)
bsw43_val = bsw43*np.sin(t)
bsw44_val = bsw44*np.sin(t)

# MADX
#-BSStren*(BSW40/BSW42)
# Equivalent
#-BSS_val[t]*(bsw40*np.sin(t) / bsw42*np.sin(t))

# We want a table, with K2 columns, one for each sextupole (4)
# 9 spaces before and between each column
# with header something like:
d = datetime.datetime.today()
date = str(str(d.day) + '/'+ str(d.month) +'/' + str(d.year))
header = str('@ NAME             %08s \"BSWTABLE\"\n@ TYPE             %04s \"USER\"\n@ TITLE            %03s \"BSW\"\n@ ORIGIN           %16s \"5.02.00 Linux 64\"\n@ DATE             %08s \"'+date+'\"\n@ TIME             %08s \"12.09.14\"\n')

f.write(header)
f.write('*'+' '.join(map(lambda i: i.rjust(18), ['BSEXT_T', 'BSS', 'BSW40_K0', 'BSW42_K0', 'BSW43_K0', 'BSW44_K0'])) + '\n')
f.write('$'+' '.join(map(lambda i: i.rjust(18), ['%le', '%le', '%le', '%le','%le', '%le'])) + '\n')

if half_bump: 
    for i in xrange(0,len(s)):
        f.write(' '+' '.join(map(lambda i: ('%1.4e'%i).rjust(18), [u[i], BSS_max*np.cos(s[i]), bsw40*np.sin(s[i]), bsw42*np.sin(s[i]), bsw43*np.sin(s[i]), bsw44*np.sin(s[i])])) + '\n')

else:
    for i in xrange(0,len(t)):
        f.write(' '+' '.join(map(lambda i: ('%1.4e'%i).rjust(18), [t[i], BSS_max*np.cos(t[i]), bsw40*np.sin(t[i]), bsw42*np.sin(t[i]), bsw43*np.sin(t[i]), bsw44*np.sin(t[i])])) + '\n')


### Figure parameters

In [17]:
plt.rcParams['figure.figsize'] = [5.0, 4.5]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 14

plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

plt.rcParams['font.size'] = 10
plt.rcParams['legend.fontsize'] = 10

plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.markersize'] = 5

### Function Declarations

In [3]:
def harmonic_fn(x, a, b, c, d):
    return a + b*np.cos(c + 2*np.pi*d*x)

def quadratic(x, a, m, c):
    return a*x**2 + x*m + c

def quadratic(x, a, m, c):
    return a*x**2 + x*m + c

def Md_fit(x, a, b, c, d):
        return a + b*np.cos(c*x+d)
    
def Md_ramp_fit(x, a, b, c, d):
    return a + b*(x-1)*np.cos(c*(x-1)+d)

def round_sig(x, sig=4):
    return round(x, sig-int(floor(log10(abs(x))))-1)

In [4]:
def add_input_file(dd, filename, label):
	f = filename
	p = dict()
	sio.loadmat(f, mdict=p)
	dd[label] = p	
	print '\tAdded output data from ', filename, '\t dictionary key: ', label
	return dd

In [9]:
def Read_PTC_Twiss_Return_Dict(filename, verbose=True):
    # Dictionary for output
    d = dict()
    keywords = ''
    
    # First we open and count header lines
    fin0=open(filename,'r').readlines()
    headerlines = 0
    for l in fin0:
        headerlines = headerlines + 1
        if '* NAME' in l:
            keywords = l
            break
    headerlines = headerlines + 1
    
    if verbose: print '\nRead_PTC_Twiss_Return_Dict found Keywords: \n',keywords
    dict_keys = []
    for key in keywords.split():
        dict_keys.append(key)
    dict_keys.remove('*')
    if verbose: print '\nRead_PTC_Twiss_Return_Dict Dict Keys: \n',dict_keys
    
    for key in dict_keys:
        d[key]=[]
        
    if verbose: print '\nRead_PTC_Twiss_Return_Dict empty dictionary \n', d
    
    # Strip header
    fin1=open(filename,'r').readlines()[headerlines:]   
    
    for l in fin1:
        i = -1        
        for value in l.split():
            i = i+1
            if 'NAME' in dict_keys[i]:
                d[dict_keys[i]].append(str(value))
            else:
                d[dict_keys[i]].append(float(value))    
                
    return dict_keys, d

#### MAD-X Function for applying single quadrupole error to QDN78

IF(quad_error==1){

        !beta_0 = 22.35981157; !value @ QDN72
        L_Q = 0.12;
        beta_0 = 21.45106468; !value @ QDN78
        value, beta_0;

        Delta_k_unit = ( (2 * sin(twopi*(6+tune_y)) ) / (beta_0 * L_Q) );
        value, Delta_k_unit;

        !Delta_k := 1.05 * ( (2 * sin(twopi*(6+tune_y)) ) / (beta_0) );
        !Delta_k := Beta_beating * 0.013781475; !For specific MD4224 optics
        !Delta_k_unit_old := 0.000510425; !For specific MD4224 optics

        Delta_k_new := -Beta_beating * 1E-2 * (Delta_k_unit*L_Q);
        value, Delta_k_new;

        EOPTION, ADD=false;
        SELECT,FLAG=ERROR,CLEAR;
        SELECT,FLAG=ERROR,PATTERN=PR.QDN78;
        EFCOMP, DKN:={0,Delta_k_new,0}; 
}

In [72]:
def dk_from_beta_beating(beta_beating_percentage, L_Q, beta_0, tune_y):
    return -beta_beating_percentage * 1E-2 * (( (2 * np.sin(2*np.pi*(tune_y)) ) / (beta_0 * L_Q) ))
    

# Import Data

In [55]:
keys_00, d_00 = Read_PTC_Twiss_Return_Dict('00/optimised_flat_file.tfs')
keys_05, d_05 = Read_PTC_Twiss_Return_Dict('05/optimised_flat_file.tfs')
keys_10, d_10 = Read_PTC_Twiss_Return_Dict('10/optimised_flat_file.tfs')


Read_PTC_Twiss_Return_Dict found Keywords: 
* NAME                                S               BETX               BETY               ALFX               ALFY              DISP1              DISP2              DISP3              DISP4                  X                 PX                  Y                 PY 


Read_PTC_Twiss_Return_Dict Dict Keys: 
['NAME', 'S', 'BETX', 'BETY', 'ALFX', 'ALFY', 'DISP1', 'DISP2', 'DISP3', 'DISP4', 'X', 'PX', 'Y', 'PY']

Read_PTC_Twiss_Return_Dict empty dictionary 
{'PY': [], 'NAME': [], 'BETY': [], 'BETX': [], 'DISP4': [], 'Y': [], 'S': [], 'PX': [], 'DISP1': [], 'X': [], 'DISP3': [], 'DISP2': [], 'ALFY': [], 'ALFX': []}

Read_PTC_Twiss_Return_Dict found Keywords: 
* NAME                                S               BETX               BETY               ALFX               ALFY              DISP1              DISP3                MUX                MUY                  X                 PX                  Y                 PY 


Read_PTC_Twiss_Retu